# Topic modeling : LDA

## Dataset : Grand Débat National (Great national debate)

The aim of this exercise is to be familiar with the text-mining and topic models such as LDA. One of the contexts where topic modeling is very useful is in open-ended questions. It allows us to explore the variation of topics addressed in people's responses. For this we will use the french "Grand Débat National" dataset. This dataset presents a complete set of responses from the [Grand Débat National](https://granddebat.fr/), the public debate organized by President Macron. The purpose of the debate was to better understand the needs and opinions of the French people following the Yellow Vests protests. The results of this debate are now available as [open data](https://granddebat.fr/pages/donnees-ouvertes).

## 1. Import data

**Question 1 :** Download one of the ecological transition csv files and load the content into a pandas dataframe. Name this variable `raw_data`

In [3]:
%load_ext lab_black

In [4]:
import pandas as pd

path = "https://raw.githubusercontent.com/curiousML/DSA/master/text_mining/data/LA_TRANSITION_ECOLOGIQUE.csv"
# "/chemin/menant/a/LA_TRANSITION_ECOLOGIQUE.csv"

# our dataframe
raw_data = pd.read_csv(path)

/var/folders/_c/p1x36y5n3dv4rpm9sgpjv0z00000gn/T/ipykernel_76053/2905575155.py:7: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_data = pd.read_csv(path)


We will focus on the last question: ``Y a-t-il d'autres points sur la transition écologique sur lesquels vous souhaiteriez vous exprimer ?`` We hope that our LDA model will help us to analyze the topics on which their responses are focused. Let's take a look on the data :

In [5]:
question = "Y a-t-il d'autres points sur la transition écologique sur lesquels vous souhaiteriez vous exprimer ?"
raw_data[question].head(10)

0               Multiplier les centrales géothermiques
1    Les problèmes auxquels se trouve confronté l’e...
2                                                  NaN
3                                                  NaN
4      Une vrai politique écologique et non économique
5    Les bonnes idées ne grandissent que par le par...
6    Pédagogie dans ce sens là dés la petite école ...
7                                                  NaN
8    faire de l'écologie incitative et non punitive...
9    Développer le ferroutage pour les poids lourds...
Name: Y a-t-il d'autres points sur la transition écologique sur lesquels vous souhaiteriez vous exprimer ?, dtype: object

As we note, there is a lot of missing data (like any open-ended question, people decide whether or not to write a comment). A cleanup step is necessary.

## 2. Clean and vectorize documents

Before training our LDA model, we need to tokenize our text. We will tokenize with the [spaCy]  (https://spacy.io/) library because we will only perform some basic preprocessing. We will just initialize a blank template for the French language.

In [5]:
!python -m spacy download fr_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 11.2 MB/s eta 0:00:0000:0100:01
DEPRECATION: nb-black 1.0.7 has a non-standard dependency specifier black>='19.3'; python_version >= "3.6". pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of nb-black or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')


In [6]:
import spacy

nlp = spacy.load("fr_core_news_sm")

# nlp = spacy.blank("fr")

Let's remove all the rows from the dataframe that don't have an answer for our question (the `NaNs above). This new dataframe will be called ``texts``.

In [7]:
texts = raw_data.dropna(subset=[question])[question]
# assert texts[question].isnull().sum() == 0

In [8]:
texts.head()

0               Multiplier les centrales géothermiques
1    Les problèmes auxquels se trouve confronté l’e...
4      Une vrai politique écologique et non économique
5    Les bonnes idées ne grandissent que par le par...
6    Pédagogie dans ce sens là dés la petite école ...
Name: Y a-t-il d'autres points sur la transition écologique sur lesquels vous souhaiteriez vous exprimer ?, dtype: object

First preprocessing with spacy :

In [9]:
spacy_docs = list(nlp.pipe(texts))

We now have a list of spaCy documents. We will transform each spaCy document into a list of tokens. Instead of the original tokens, we will work with lemmas instead. This will allow our model to generalize better

Here is the full list of preprocessing used: 
 
- remove all **words less than 3 characters**,
- remove all **stop-words**, and
- lemmatize** the remaining words and,
- put these words in **minuscule**.

In [10]:
docs = [
    [t.lemma_.lower() for t in doc if len(t.orth_) > 3 and not t.is_stop]
    for doc in spacy_docs
]
print(docs[:3])

[['multiplier', 'centrale', 'géothermique'], ['problème', 'trouver', 'confronté', 'ensemble', 'planète', 'dénoncer', 'parfaire', 'désordre', 'gilet', 'jaune', 'france', 'il', 'surpopulation', 'mondial', 'population', 'passer', 'd’1,5', 'milliard', 'habitant', '1900', 'milliard', '2020', 'monter', 'bientôt', 'milliard', '2040', 'progrès', 'communication', 'village', 'mondial', 'individu', 'fond', 'asie', 'fond', 'afrique', 'passer', 'quartier', 'campagne', 'pays', 'aspirer', 'vivre', 'blâmer', 'lotir', 'concitoyen', 'logement', 'nourriture', 'bien', 'consommation', 'déplacemer', 'etc.', 'mère', 'problème', 'bien', 'solution', 'problème', 'stabilisation', 'croissance', 'démographique', 'partage', 'richesse', 'partage', 'terre', 'partage', 'protection', 'biodiversité', 'règlement', 'conflit', 'lutte', 'contre', 'déforestation', 'lutte', 'contre', 'dérèglement', 'climatique', 'règlement', 'conflit', 'stabilisation', 'migration', 'concurrence', 'commercial', 'mondial', 'etc.', 'français', '

In order to preserve some word order in our modeling, we will take into account frequent bigrams. For this, we will use the [Gensim](https://radimrehurek.com/gensim/)library. We would like to point out that the Gensim library is an excellent NLP library for topics modeling. 

Here is the chosen process: 

- We first identify the frequent bigrams in the corpus, 
- then we add them to the list of tokens for the documents in which they appear. This means that the bigrams will not be in their correct position in the text, but this is not a problem: topic models are bag-of-words models that ignore the position of words anyway.

In [11]:
import re
from gensim.models import Phrases

bigram = Phrases(docs, min_count=10)

for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if (
            "_" in token
        ):  # les bigrammes peuvent être reconnus par "_" qui concatène les mots
            docs[idx].append(token)

Let's move on to the last steps of the Gensim specific preprocessing. First, we will create a dictionary representation of the documents. This dictionary will map each word to a unique identifier and will help us create word-sack representations of each document. These bag-of-words representations contain the identifiers of the words in the document as well as their frequency. In addition, we can remove the least frequent and most frequent words from the vocabulary. This will improve the quality of our model and speed up its training. The minimum frequency of a word is expressed as an absolute number, the maximum frequency is the proportion of documents in which a word can appear.

In [12]:
from gensim.corpora import Dictionary

dictionary = Dictionary(docs)
print("Number of unique words in original documents :", len(dictionary))

dictionary.filter_extremes(no_below=3, no_above=0.25)
print("Number of unique words after removing rare and common words :", len(dictionary))

print("Example representation of document 3 :", dictionary.doc2bow(docs[2]))

Number of unique words in original documents : 32371
Number of unique words after removing rare and common words : 12457
Example representation of document 3 : [(86, 1), (87, 1), (88, 1), (89, 1)]


Next, we create bag-of-words representations for each document in the corpus see method [doc2bow](https://radimrehurek.com/gensim/corpora/dictionary.html) :

In [13]:
corpus = [dictionary.doc2bow(doc) for doc in docs]

## 3. Topic Modeling with LDA

Now it's time to train our LDA! To do this, we use the following parameters: 

- **corpus**: the bag-of-words representations of our documents
- **id2token**: the mapping of indexes to words
- **num_topics** : the number of topics the model should identify (let's set <font color = "red"><b>10</b></font>)
- **chunksize**: the number of documents the model sees on each update (let's set to <font color = "red"><b>1,000</b></font>)
- **passes**: the number of times we show the total corpus to the model during training (let's set to <font color = "red"><b>5</b></font>)
- **random_state**: we use a seed to ensure reproducibility (let's set to <font color = "red"><b>1</b></font>)

On a corpus of this size, training usually takes one or two minutes.

**Question 2 :**

In [14]:
from gensim.models import LdaModel

model = LdaModel(
    corpus, num_topics=10, id2word=dictionary, chunksize=1000, passes=5, random_state=1
)

## 4. Results and visualization

**Question 3 :** Let's see what the model has learned. To do this, let's display the ten most characteristic words for each topic.

In [15]:
for topic, words in model.print_topics():
    print("***********")
    print("* topic", topic + 1, "*")
    print("***********")
    print(topic + 1, ":", words)
    print()

***********
* topic 1 *
***********
1 : 0.080*"produit" + 0.021*"emballage" + 0.019*"plastique" + 0.018*"déchet" + 0.015*"interdir" + 0.012*"local" + 0.011*"interdire" + 0.011*"taxer" + 0.011*"pesticide" + 0.011*"consommation"

***********
* topic 2 *
***********
2 : 0.031*"taxer" + 0.029*"taxe" + 0.025*"transport" + 0.017*"pollueur" + 0.015*"camion" + 0.014*"avion" + 0.013*"route" + 0.011*"carburant" + 0.011*"vitesse" + 0.011*"lourd"

***********
* topic 3 *
***********
3 : 0.028*"ville" + 0.026*"transport" + 0.017*"grand" + 0.013*"centre" + 0.013*"zone" + 0.012*"commun" + 0.010*"petit" + 0.009*"local" + 0.009*"urbain" + 0.008*"commerce"

***********
* topic 4 *
***********
4 : 0.051*"écologique" + 0.047*"transition" + 0.029*"transition_écologique" + 0.022*"falloir" + 0.018*"faire" + 0.013*"citoyen" + 0.012*"politique" + 0.010*"entreprise" + 0.009*"solution" + 0.009*"terme"

***********
* topic 5 *
***********
5 : 0.018*"énergie" + 0.016*"solaire" + 0.015*"panneau" + 0.014*"investisse

Another way to observe topics is to **visualize** them. This can be done with the library [pyLDAvis](https://github.com/bmabey/pyLDAvis). PyLDAvis will show us how popular the topics are in our corpus, how similar the topics are, and which words are most important for that topic. Note that it is important to set ``sort_topics = False`` on the call to pyLDAvis. If you don't, the topics will be sorted differently than in Gensim. This may take a few minutes to load.

**Question 5 :**

In [17]:
! pip install pyLDAvis

DEPRECATION: nb-black 1.0.7 has a non-standard dependency specifier black>='19.3'; python_version >= "3.6". pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of nb-black or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [32]:
import pyLDAvis.gensim_models
import warnings

pyLDAvis.enable_notebook()
warnings.filterwarnings("ignore", category=DeprecationWarning)

pyLDAvis.gensim_models.prepare(model, corpus, dictionary, sort_topics=False)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.251655  0.055009       1        1   7.438676
1     -0.110234 -0.122120       2        1   8.644465
2     -0.101331 -0.141036       3        1   8.068318
3      0.192782  0.044430       4        1  13.075055
4     -0.026788 -0.171747       5        1   9.433054
5     -0.212940  0.255568       6        1   6.689580
6      0.168914  0.018892       7        1  18.948061
7      0.133260 -0.044589       8        1  11.489400
8      0.048377 -0.119033       9        1  10.377123
9      0.159616  0.224627      10        1   5.836267, topic_info=           Term         Freq        Total Category  logprob  loglift
543     produit  4552.000000  4552.000000  Default  30.0000  30.0000
466  transition  4727.000000  4727.000000  Default  29.0000  29.0000
88   écologique  6110.000000  6110.000000  Default  28.0000  28.0000
256     énergie  5336.000000  5336.000000  Default  27.0000  27.0000
169  électrique  3106.000000  3106.000000  Default  26.0000  26.0000
..          ...          ...          ...      ...      ...      ...
140       monde   295.667290  1557.315871  Topic10  -5.0202   1.1796
867        agir   271.360615   892.477674  Topic10  -5.1060   1.6505
306       chose   261.931496   738.361612  Topic10  -5.1413   1.8047
40     français   282.435028  2428.344367  Topic10  -5.0660   0.6896
567     vouloir   268.689293  1257.260518  Topic10  -5.1159   1.2979

[653 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
9489      6  0.989577      \n\n  
8633      8  0.995185     \n     
5709     10  0.994558     .......
1505      1  0.030165  abandonner
1505      2  0.034474  abandonner
...     ...       ...         ...
1258      6  0.030848        être
1258      7  0.478681        être
1258      8  0.153178        être
1258      9  0.062760        être
1258     10  0.074993        être

[1466 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

Finally, let's look at the topics that the model recognizes in some of the individual documents. Here we see how LDA tends to assign a high probability to a small number of topics for each document, making its results highly interpretable.

In [33]:
# Nous en affichons que 8
i = 0
for text, doc in zip(texts[:8], docs[:8]):
    i += 1
    print("***********")
    print("* doc", i, "  *")
    print("***********")
    print(text)
    print(
        [
            (topic + 1, prob)
            for (topic, prob) in model[dictionary.doc2bow(doc)]
            if prob > 0.1
        ]
    )
    print()

***********
* doc 1   *
***********
Multiplier les centrales géothermiques
[(8, 0.7749159)]

***********
* doc 2   *
***********
Les problèmes auxquels se trouve confronté l’ensemble de la planète et que dénoncent, dans le plus parfait désordre, les gilets jaunes de France ne sont-ils pas dus, avant tout, à la surpopulation mondiale ? Cette population est passée d’1,5 milliards d’habitants en 1900 à 7 milliards en 2020 et montera bientôt à 10 milliards vers 2040.  Avec les progrès de la communication dans ce village mondial, chaque individu, du fin fond de l’Asie au fin fond de l’Afrique, en passant par les « quartiers » et les « campagnes » de notre pays, aspire à vivre – et on ne peu l’en blâmer – comme les moins mal lotis de nos concitoyens (logement, nourriture, biens de consommation, déplacement,etc.).  Voilà la mère de tous les problèmes. Si tel est bien le cas, la solution à tous les problèmes (stabilisation de la croissance démographique, partage des richesses, partage des terr

Many collections of unstructured text are not accompanied by labels. Topic models such as LDA are a useful technique for discovering the most important topics in these documents. **Gensim** facilitates learning about these topics and **pyLDAvis** presents the results in a visually appealing way. Together, they form a powerful toolkit for better understanding what's inside large document sets and for exploring subsets of related texts. While these results are often already quite revealing, it is also possible to use them as a starting point, for example, for a labeling exercise for supervised text classification. In sum, thematic models should be in every data scientist's toolbox as a very quick way to gain insight into large document collections.